# Петров Сергей

# Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import math

from sklearn.model_selection import train_test_split
from scipy.special import softmax

In [2]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [3]:
r_state = 42

# Подготовка датасетов

In [26]:
adult_df = pd.read_csv('adult.csv')

In [27]:
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [28]:
adult_train_df, adult_test_df = train_test_split(adult_df,  test_size=0.25, random_state=r_state)

In [29]:
adult_train_df.groupby(['sex']).get_group('Male')

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
29,38,Private,175360,10th,6,Never-married,Prof-specialty,Not-in-family,White,Male,0,2559,90,United-States,>50K
12181,25,Private,131463,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<=50K
18114,63,Self-emp-not-inc,181561,Some-college,10,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,50,United-States,<=50K
4278,50,Federal-gov,251585,Bachelors,13,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,55,United-States,>50K
12050,23,?,145964,Some-college,10,Never-married,?,Not-in-family,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,25,Private,410240,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
5390,51,Private,146767,Assoc-voc,11,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
860,55,Federal-gov,238192,HS-grad,9,Married-civ-spouse,Tech-support,Husband,White,Male,0,1887,40,United-States,>50K
15795,41,Private,154076,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,50,United-States,>50K


In [30]:
from sklearn.datasets import load_iris

In [31]:
iris_data = load_iris(as_frame=True)

In [32]:
iris_df = iris_data['data']
iris_df['target'] = iris_data['target']

In [33]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [34]:
train_df, test_df = train_test_split(iris_df, test_size=0.5, random_state=42)

In [35]:
x_cols = iris_df.columns.to_list()
x_cols.remove('target')

In [36]:
x_cols

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [37]:
X_train = train_df[x_cols]
y_train = train_df['target']

X_test = test_df[x_cols]
y_test = test_df['target']

# Наивный Байесовский классификатор для непрервыных фичей(гауссиан)

In [16]:
def gausFunc(x, m, d):
    # return math.exp(-(x - m)**2/(2*d**2))/math.sqrt(2*math.pi*d**2)
    return np.exp(-(x - m) ** 2 / (2 * d ** 2)) / np.sqrt(2 * np.pi * d ** 2)


class MyGaussianNB:
    def __init__(self):
        
        # Массив средних значений для каждой фичи, для каждого класса
        # (размер NxK, где N - кол-во фичей, K - кол-во классов)
        self.mus = {}

        # Массив стандартных среднеквадратических отклоенний для каждой фичи
        # для каждого класса (размер NxK, где N - кол-во фичей, K - кол-во классов)
        self.sigmas = {}
        
        self.p = {}
        
        self.coluns = []
        self.costil = {}
    
    def fit(self, X, y):
        df = X.copy()
        df['target'] = y
        
        self.columns = y.unique()
        grouped = df.groupby(['target'])
        
        for u in self.columns:
            group = grouped.get_group(u).select_dtypes(include=np.number)
            nu = str(u)
            self.costil[nu] = u
            
            self.mus[nu] = group.mean()
            self.sigmas[nu] = group.std()
            self.p[nu] = group.shape[0]/X.shape[0]
        
        
    def predict(self, X):
        return self.likelihood(X).idxmax(axis=1).apply(lambda x : self.costil[x])
    
    def predict_proba(self, X):
        # Применить функцию softmax (scipy.special.softmax) на выходы likelihood
        return self.likelihood(X).apply(lambda x: softmax(x))
    
    def likelihood(self, X):
        # Для каждого класса посчитать функции правдоподобия для каждой фичи
        # Прологарифмировать и сложить их между собой и добавить логарифм
        # априорной вероятности соответствующего класса
        # Получится матрица размером NxK, где N - кол-во строк, K - кол-во классов
        df = X.select_dtypes(include=np.number)
        categ = [* self.mus]
        features = df.columns
        
        for cat in categ:
            df[cat] = sum([np.log(gausFunc(df[feat],self.mus[cat][feat], self.sigmas[cat][feat] ))for feat in features]) + np.log(self.p[cat])
            #print([np.log(gausFunc(df[feat],self.mus[cat][feat], self.sigmas[cat][feat] ))for feat in features])
        return df[categ]

In [17]:
gnb = MyGaussianNB()

In [18]:
gnb.fit(X_train, y_train)

In [19]:
gnb.likelihood(X_test)

,1,2,0
73,-1.327471,-6.637869,-256.689725
18,-31.871922,-60.222791,-3.479330
118,-28.668378,-9.290720,-798.078356
78,-1.255465,-5.454553,-256.506249
76,-2.492752,-4.595382,-294.339389
...,...,...,...
113,-7.331660,-3.985124,-382.903750
33,-43.859258,-74.085996,-2.661841
138,-4.187132,-2.876021,-328.702924
101,-5.702914,-2.638833,-383.524409


In [20]:
iris_gnb = gnb.predict(X_test)

In [21]:
gnb.predict_proba(X_test)

,1,2,0
73,6.564663e-02,5.659475e-04,1.374757e-113
18,3.563900e-15,3.027893e-27,1.276742e-03
118,8.774194e-14,3.987106e-05,0.000000e+00
78,7.054788e-02,1.847924e-03,1.651615e-113
76,2.047092e-02,4.363320e-03,6.126107e-130
...,...,...,...
113,1.620414e-04,8.032467e-03,2.109460e-168
33,2.217644e-20,2.886868e-33,2.891573e-03
138,3.760776e-03,2.435153e-02,7.299463e-145
101,8.260030e-04,3.086992e-02,1.134025e-168


In [25]:
gnb.predict_proba(X_test).sum()

1    1.0
2    1.0
0    1.0
dtype: float64

In [22]:
np.unique(iris_gnb == y_test, return_counts = True)

(array([False,  True]), array([ 1, 74]))

In [23]:
np.unique(gnb.predict(X_train) == y_train, return_counts = True)

(array([False,  True]), array([ 6, 69]))

In [24]:
accuracy_score(y_train, gnb.predict(X_train)), accuracy_score(y_test, gnb.predict(X_test))

(0.92, 0.9866666666666667)

# Наивный баесовский классификатор для категориальных фичей